In [1]:
# TFDS Image Transformations (diagrams) 

import os
import tensorflow as tf
import tensorflow_datasets as tfds

# Settings
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3
OUTPUT_DIR = "diagrams_ch3"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load TFDS flowers dataset
ds, info = tfds.load("tf_flowers", split="train", with_info=True, shuffle_files=True)

# Take one example
for example in ds.take(1):
    img = tf.image.convert_image_dtype(example["image"], tf.float32)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    label = example["label"].numpy()
    break

# Helper: mean filter using avg_pool2d
def mean_filter(image, filter_size):
    image = tf.expand_dims(image, axis=0)  # add batch dim
    filtered = tf.nn.avg_pool2d(image, ksize=filter_size, strides=1, padding='SAME')
    return tf.squeeze(filtered, axis=0)  # remove batch dim

# 1. Original
original = img

# 2. Smooth
smooth = mean_filter(img, filter_size=11)

# 3. Edge (difference of smoothed images)
edge = mean_filter(img, filter_size=5) - mean_filter(img, filter_size=11)
edge = tf.clip_by_value(tf.image.rgb_to_grayscale(edge), 0, 1)

# 4. Inverted grayscale intensity
intensity = 1.0 - tf.image.rgb_to_grayscale(img)

# Helper: save image to PNG
def save_image(tensor, filename):
    tensor_uint8 = tf.image.convert_image_dtype(tensor, tf.uint8)
    encoded = tf.image.encode_png(tensor_uint8)
    tf.io.write_file(filename, encoded)

# Save all diagrams
save_image(original, os.path.join(OUTPUT_DIR, "original.png"))
save_image(smooth, os.path.join(OUTPUT_DIR, "smooth.png"))
save_image(edge, os.path.join(OUTPUT_DIR, "edge.png"))
save_image(intensity, os.path.join(OUTPUT_DIR, "intensity.png"))

print(f"Saved diagram images to folder: {OUTPUT_DIR}")


Saved diagram images to folder: diagrams_ch3
